In [2]:
!curl https://download.mosek.com/stable/8.1.0.82/mosektoolslinux64x86.tar.bz2 --output /root/mosektoolslinux64x86.tar.bz2
!cd /root/; tar xvf /root/mosektoolslinux64x86.tar.bz2
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/Mosek/mosek.lic /root/mosek/
!pip install git+https://github.com/convexengineering/gpkit
!pip show gpkit

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.3M  100 40.3M    0     0  90.3M      0 --:--:-- --:--:-- --:--:-- 90.3M
mosek/8/tools/examples/python/parameters.py
mosek/8/toolbox/examples/cqo1.m
mosek/8/toolbox/r2014a/mskqpopt.m
mosek/8/tools/examples/fusion/java/breaksolver.java
mosek/8/toolbox/r2014a/mskcheck.m
mosek/8/tools/examples/fusion/cxx/lo1.cc
mosek/8/tools/examples/data/dgo.f
mosek/8/tools/examples/dotnet/scopt1.cs
mosek/8/tools/examples/dotnet/solvebasis.cs
mosek/8/toolbox/r2014aom/mskerrmsg.m
mosek/8/toolbox/examples/simple.m
mosek/8/tools/examples/java/solutionquality.java
mosek/8/tools/examples/fusion/dotnet/production.cs
mosek/8/tools/examples/fusion/java/Makefile
mosek/8/tools/examples/fusion/python/lo1.py
mosek/8/tools/examples/fusion/java/duality.java
mosek/8/tools/platform/linux64x86/bin/libcilkrts.so.5
mosek/8/toolbox/examples/qo2.m
mosek/8/tools/ex

In [3]:
from pylab import *
from gpkit import VectorVariable, Variable, Model, units

Found no installed solvers, beginning a build.
# Building GPkit version 1.1
# Moving to the directory from which GPkit was imported.

Attempting to find and build solvers:

# Looking for `mosek_cli`
#   (A "success" is if mskexpopt complains that
#    we haven't specified a file for it to open.)
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##
# Looks like `mskexpopt` was not found in the default PATH,
#  so let's try locating that binary ourselves.
#   Adding /root/mosek/8/tools/platform/linux64x86/bin to the PATH
#     Calling 'mskexpopt'
##
### CALL BEGINS
### CALL ENDS
##

Found mosek_cli in /root/mosek/8/tools/platform/linux64x86/bin

# Looking for `mosek_conif`
#   Trying to import mosek...
# Did not find
# mosek_conif

# Looking for `cvxopt`
#   Trying to import cvxopt...

Found cvxopt in the default PYTHONPATH

Found the following solvers: mosek_cli, cvxopt
#     Replacing directory env


..........................E...................................................................................................................................../usr/local/lib/python3.11/dist-packages/gpkit/constraints/sgp.py:177: UserWarning: SGP not convergent: Cost rose by 0.0099% (9.99901e-31 to 1e-30) on GP solve 2. Details can be found in `m.program.results` or by solving at a higher verbosity. Note convergence is not guaranteed for models with SignomialEqualities.
  pywarnings.warn(
.................................................
ERROR: test_vector_sweep (gpkit.tests.t_sub.TestModelSubs.test_vector_sweep)
Test sweep involving VectorVariables
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gpkit/tests/t_sub.py", line 205, in test_vector_sweep
    sol = m.solve(verbosity=0)
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gpkit/constraints/prog_f


GPkit is now installed with solver(s) ['mosek_cli', 'cvxopt']
To incorporate new solvers at a later date, run `gpkit.build()`.

If any tests didn't pass, please post the output above
(starting from "Found no installed solvers, beginning a build.")
to gpkit@mit.edu or https://github.com/convexengineering/gpkit/issues/new
so we can prevent others from having these errors.

The same goes for any other bugs you encounter with GPkit:
send 'em our way, along with any interesting models, speculative features,
comments, discussions, or clarifications you feel like sharing.

Finally, we hope you find our documentation (https://gpkit.readthedocs.io/)
and engineering-design models (https://github.com/convexengineering/gplibrary/)
to be useful resources for your own applications.

Enjoy!



In [25]:
import datetime
import re, sys
import networkx as nx
import numpy as np
class Gate:
    def __init__(self, name, gate_type,p=0,l=0):
        self.name, self.gate_type, self.inputs, self.outputs = name, gate_type, [], []
        self.p = p
        self.l = l

    def add_input(self, gate):
        if gate not in self.inputs:
            self.inputs.append(gate)
            if self not in gate.outputs: gate.outputs.append(self)

    def add_output(self, gate):
        if gate not in self.outputs:
            self.outputs.append(gate)
            if self not in gate.inputs: gate.inputs.append(self)

class Net:
    def __init__(self, name, source=None, destinations=None):
        self.name, self.source, self.destinations, self.value = name, source, destinations or [], None

    def add_destination(self, gate):
        if gate not in self.destinations: self.destinations.append(gate)

    def set_source(self, gate):
        self.source = gate

class Circuit:
    def __init__(self):
        self.gates, self.nets, self.inputs, self.outputs = {}, {}, [], []
        self.gate_order = []

    def add_gate(self, gate):
        self.gates[gate.name] = gate
        if gate.name not in self.gate_order:
            self.gate_order.append(gate.name)

    def add_net(self, net): self.nets[net.name] = net

    def connect(self, source_gate_name, dest_gate_name, net_name=None):
        source_gate, dest_gate = self.gates.get(source_gate_name), self.gates.get(dest_gate_name)
        if not source_gate or not dest_gate: raise ValueError(f"Cannot connect: gate not found")

        net_name = net_name or f"{source_gate_name}_to_{dest_gate_name}"
        if net_name not in self.nets:
            net = Net(net_name, source_gate, [dest_gate])
            self.add_net(net)
        else:
            net = self.nets[net_name]
            net.set_source(source_gate)
            net.add_destination(dest_gate)

        source_gate.add_output(dest_gate)
        dest_gate.add_input(source_gate)
        return net

    def set_input_gates(self, gate_names): self.inputs = gate_names

    def set_output_gates(self, gate_names): self.outputs = gate_names

    def get_fanouts(self):
        return {gate_name: [g.name for g in gate.outputs] for gate_name, gate in self.gates.items()}
    def get_all_paths(self):
        sources = [gn for gn, g in self.gates.items() if not g.inputs]
        sinks = [gn for gn, g in self.gates.items() if not g.outputs]
        all_paths = []

        def dfs(current, path):
            path.append(current)
            if current in sinks:
                all_paths.append(path.copy())
            else:
                for next_gate in self.gates[current].outputs:
                    dfs(next_gate.name, path)
            path.pop()

        for source in sources:
            dfs(source, [])

        return all_paths
    def get_longest_paths(self):
        sources = [gn for gn, g in self.gates.items() if not g.inputs]
        sinks = [gn for gn, g in self.gates.items() if not g.outputs]
        longest_paths, max_length = [], 0

        def find_paths(current, path, visited):
            nonlocal longest_paths, max_length
            visited.add(current); path.append(current)

            if current in sinks:
                path_length = len(path)
                if path_length > max_length: max_length, longest_paths = path_length, [path.copy()]
                elif path_length == max_length: longest_paths.append(path.copy())

            for next_gate in self.gates[current].outputs:
                if next_gate.name not in visited: find_paths(next_gate.name, path, visited)

            path.pop(); visited.remove(current)

        for source in sources: find_paths(source, [], set())
        return longest_paths, max_length

def parse(file_contents):
    circuit = Circuit()
    patterns = {
        'input': re.compile(r'input\s+([^;]+);'),
        'output': re.compile(r'output\s+([^;]+);'),
        'wire': re.compile(r'wire\s+([^;]+);'),
        'gate': re.compile(r'(not|nand|nor)\s+(\w+)\s*\(([^)]+)\);', re.MULTILINE)
    }

    inputs = [inp.strip() for inp in patterns['input'].search(file_contents).group(1).split(',')] if patterns['input'].search(file_contents) else []
    outputs = [out.strip() for out in patterns['output'].search(file_contents).group(1).split(',')] if patterns['output'].search(file_contents) else []
    wires = [wire.strip() for wire in patterns['wire'].search(file_contents).group(1).split(',')] if patterns['wire'].search(file_contents) else []

    for signal_name in inputs + outputs + wires: circuit.add_net(Net(signal_name))

    for gate_match in patterns['gate'].finditer(file_contents):
        gate_type, gate_name = gate_match.group(1).upper(), gate_match.group(2)
        if not any(gate_name.lower().startswith(prefix) for prefix in ['nand', 'nor', 'not']): continue

        ports = [port.strip() for port in gate_match.group(3).split(',')]
        output_port, input_ports = ports[0], ports[1:]

        gate = Gate(gate_name, gate_type)
        circuit.add_gate(gate)

        output_net = circuit.nets.get(output_port) or Net(output_port)
        if output_port not in circuit.nets: circuit.add_net(output_net)
        output_net.set_source(gate)

        for input_port in input_ports:
            input_net = circuit.nets.get(input_port) or Net(input_port)
            if input_port not in circuit.nets: circuit.add_net(input_net)
            input_net.add_destination(gate)

        if output_port in outputs and circuit.gates.get(output_port): circuit.connect(gate_name, output_port)

    for net_name, net in circuit.nets.items():
        if net.source and net.destinations:
            source_gate = net.source
            for dest_gate in net.destinations:
                if source_gate.name != dest_gate.name:
                    is_gate = lambda g: any(g.name.lower().startswith(p) for p in ['nand', 'nor', 'not'])
                    if is_gate(source_gate) and is_gate(dest_gate): circuit.connect(source_gate.name, dest_gate.name, net_name)

    circuit.set_input_gates(inputs)
    circuit.set_output_gates(outputs)
    return circuit

def gates_with_primary_inputs(circuit):
    result = []
    for net_name in circuit.inputs:
        net = circuit.nets.get(net_name)
        if net:
            for gate in net.destinations:
                result.append(gate.name)
    return sorted(set(result))

def gates_with_primary_outputs(circuit):
    result = []
    for net_name in circuit.outputs:
        net = circuit.nets.get(net_name)
        if net and net.source:
            result.append(net.source.name)
    return sorted(set(result))


def analyze_circuit(ckt):
    def get_gate_number(gate_name):
        parts = gate_name.split('_')
        if len(parts) < 2:
            return float('inf')
        num_part = parts[1]
        return int(num_part)

    fanouts_dict = {}
    fanouts = ckt.get_fanouts()
    for gate_name in sorted(fanouts.keys(), key=get_gate_number):
        gate_fanouts = fanouts[gate_name]
        sorted_fanouts = sorted(gate_fanouts, key=get_gate_number) if gate_fanouts else []
        fanouts_dict[gate_name] = sorted_fanouts

    critical_paths_dict = {
        "max_length": 0,
        "paths": []
    }
    longest_paths, max_length = ckt.get_longest_paths()
    if longest_paths:
        critical_paths_dict["max_length"] = max_length
        def get_path_priority(path):
            if not path: return float('inf')
            return get_gate_number(path[0])
        sorted_paths = sorted(longest_paths, key=get_path_priority)
        critical_paths_dict["paths"] = sorted_paths
    return fanouts_dict, critical_paths_dict

logicalEffortDict = {
    ("NAND", 2): 4/3,
    ("NAND", 3): 5/3,
    ("NAND", 4): 2,
    ("NOR", 2): 5/3,
    ("NOR", 3): 7/3,
    ("NOR", 4): 3,
    ("INV", 1): 1
}

cload = 1000
max_gate_size = 64
def get_gate_type_and_inputs(gate_name):
    gatenum = gate_name.split("_")[1]
    prefix = gate_name.split("_")[0]
    gate_type = prefix[:-1]
    num_inputs = int(prefix[-1])
    return gate_type.upper(), num_inputs, gatenum

from gpkit import Variable, VectorVariable, Model
import math
def compute_global_tmin(fanouts_dict, critical_paths_dict, inpGates, all_paths):
    N = len(fanouts_dict)
    Gsize = VectorVariable(N, "x", sign="positive")
    noofCP = len(all_paths)
    Tmin = VectorVariable(noofCP, "T0", sign = "positive")

    g = [1.0] * N
    p = [1.0] * N
    gate_index_map = {}
    for gate in fanouts_dict:
        gate_type, num_inputs, _ = get_gate_type_and_inputs(gate)
        idx = int(gate.split("_")[1]) - 1
        gate_index_map[gate] = idx
        g[idx] = logicalEffortDict.get((gate_type, num_inputs), 1.0)
        p[idx] = num_inputs

    constraints = []

    for i, path in enumerate(all_paths):
        delay_expr = 0
        for gate in path:
            idx = gate_index_map[gate]
            delay_expr += p[idx]
            for fanout in fanouts_dict[gate]:
                f_idx = gate_index_map[fanout]
                delay_expr += g[f_idx] * (Gsize[f_idx]) / Gsize[idx]
        last_gate_idx = gate_index_map[path[-1]]
        delay_expr += cload / Gsize[last_gate_idx]
        constraints.append(Tmin[i] >= delay_expr)

    for net in circuit.nets:
      s = 0
      for gate in circuit.nets[net].destinations:
        if gate.name in inpGates and circuit.nets[net].source == None:
          idx = gate_index_map[gate.name]
          s += (g[idx] * Gsize[idx])
      if s != 0:
        print(s)
        constraints.append(s <= 50)
    # for gateName in inpGates:
    #     idx = gate_index_map[gateName]
    #     constraints.append(g[idx] * Gsize[idx] <= 50)

    constraints += [Gsize >= 1, Gsize <= max_gate_size]

    Tmax = Variable("Tmax")
    constraints += [Tmax >= T for T in Tmin]
    objective = Tmax
    m = Model(objective, constraints)
    sol = m.solve(verbosity=0)
    GsizeList = []
    for x in sol["variables"]["x"]:
     GsizeList.append(round(float(x),3))
    print(GsizeList)
    t = []
    for i, path in enumerate(all_paths):
      print(f"\nCritical Path {i + 1}: {' -> '.join(path)}")
      total_delay = 0
      for gate in path:
        idx = gate_index_map[gate]
        size = GsizeList[idx]
        parasitic = p[idx]
        if gate != path[-1]:
            s = 0
            for fo in fanouts_dict[gate]:
                s += g[gate_index_map[fo]] * GsizeList[gate_index_map[fo]]
            logical = s / size
        else:
            logical = cload / GsizeList[gate_index_map[path[-1]]]
        gate_delay = parasitic + logical
        total_delay += gate_delay
      t.append(total_delay)
      # t.append(math.floor(total_delay * 100) / 100)
    print(t)
    print(f"Normalized Optimal Twall: {sol['cost']:.2f}")
    # print("Optimized sizes:", (f"{x:.2f}" for x in list(sol["variables"]["x"])))
    # print("Sizes:",list(sol["variables"]["x"]))
    # print(max(sol["variables"]["T0"]))

    # print(sol("x").tolist())
    return round(sol['cost'], 2), GsizeList, t



# with open("sys.argv[1]", 'r') as f: file_contents = f.read()
file_contents = '''// Verilog
// c17
// Ninputs 5
// Noutputs 2
// NtotalGates 6
// NAND2 6

module c17 (N1,N2,N3,N6,N7,N22,N23);

input N1,N2,N3,N6,N7;

output N22,N23;

wire N10,N11,N16,N19;

nand NAND2_1 (N10, N1, N3);
nand NAND2_2 (N11, N3, N6);
nand NAND2_3 (N16, N2, N11);
nand NAND2_4 (N19, N11, N7);
nand NAND2_5 (N22, N10, N16);
nand NAND2_6 (N23, N16, N19);

endmodule
'''
circuit = parse(file_contents)
inpGates = gates_with_primary_inputs(circuit)
all_paths = circuit.get_all_paths()
fanouts_dict, critical_paths_dict=analyze_circuit(circuit)
Twall, GsizesList, slack = compute_global_tmin(fanouts_dict, critical_paths_dict, inpGates, all_paths)

def minimum_area(fanouts_dict, critical_paths_dict, Tspec, alpha):
  N = len(fanouts_dict)
  Gsize = VectorVariable(N, "x", sign="positive")
  noofCP = len(all_paths)
  Tmin = VectorVariable(noofCP, "T0", sign = "positive")

  g = [1.0] * N
  p = [1.0] * N
  gate_index_map = {}
  for gate in fanouts_dict:
      gate_type, num_inputs, _ = get_gate_type_and_inputs(gate)
      idx = int(gate.split("_")[1]) - 1
      gate_index_map[gate] = idx
      g[idx] = logicalEffortDict.get((gate_type, num_inputs), 1.0)
      p[idx] = num_inputs

  constraints = []

  for i, path in enumerate(all_paths):
      delay_expr = 0
      for gate in path:
          idx = gate_index_map[gate]
          delay_expr += p[idx]
          for fanout in fanouts_dict[gate]:
              f_idx = gate_index_map[fanout]
              delay_expr += g[f_idx] * (Gsize[f_idx]) / Gsize[idx]
      last_gate_idx = gate_index_map[path[-1]]
      delay_expr += cload / Gsize[last_gate_idx]
      constraints.append(Tmin[i] >= delay_expr)
      constraints.append(Tmin[i] <= Tspec)
  # print(inpGates)
  for net in circuit.nets:
    s = 0
    for gate in circuit.nets[net].destinations:
      if gate.name in inpGates and circuit.nets[net].source == None:
        idx = gate_index_map[gate.name]
        s += (g[idx] * Gsize[idx])
    if s != 0:
      print(s)
      constraints.append(s <= 50)
  # for gateName in inpGates:
  #     idx = gate_index_map[gateName]
  #     constraints.append(g[idx] * Gsize[idx] <= 50)

  constraints += [Gsize >= 1, Gsize <= max_gate_size]

  objective = sum(Gsize)

  m = Model(objective, constraints)
  print(list(m.flat()))
  sol = m.solve(verbosity=0)
  print("Alpha: ",alpha)
  print(f"Minimum Area: {sol['cost']:.2f}")
  print("Optimized sizes:", list(f"{x:.1f}" for x in sol("x")))
  # print("Sizes:",list(float(sol["variables"]["x"])))

  print(max(sol["variables"]["T0"]))
  return sol['cost'], sol("x").tolist()

# def slackBasedAreaMinimization(Twall,slack):
#   slack = [round(Twall-x,2) for x in slack]
#   print(slack)

# slackBasedAreaMinimization(Twall,slack)
# print(slack)
# print(Twall)
for alpha in [1.05, 1.1, 1.2, 1.3, 1.4, 1.5]:
    minimum_area(fanouts_dict, critical_paths_dict, Twall*alpha, alpha)

1.33·x[0]
1.33·x[2]
1.33·x[0] + 1.33·x[1]
1.33·x[1]
1.33·x[3]
[9.268, 28.232, 37.5, 18.75, 64.0, 64.0]

Critical Path 1: NAND2_1 -> NAND2_5

Critical Path 2: NAND2_2 -> NAND2_3 -> NAND2_5

Critical Path 3: NAND2_2 -> NAND2_3 -> NAND2_6

Critical Path 4: NAND2_2 -> NAND2_4 -> NAND2_6
[28.832308300963888, 28.832671043103176, 28.832671043103176, 28.832671043103176]
Normalized Optimal Twall: 28.83
1.33·x[0]
1.33·x[2]
1.33·x[0] + 1.33·x[1]
1.33·x[1]
1.33·x[3]
[gpkit.PosynomialInequality(T0[0] ≥ 2 + 1.33·x[4]/x[0] + 2 + 1e+03/x[4]), gpkit.PosynomialInequality(T0[0] ≤ 30.3), gpkit.PosynomialInequality(T0[1] ≥ 2 + 1.33·x[2]/x[1] + 1.33·x[3]/x[1] + 2 + 1.33·x[4]/x[2] + 1.33·x[5]/x[2] + 2 + 1e+03/x[4]), gpkit.PosynomialInequality(T0[1] ≤ 30.3), gpkit.PosynomialInequality(T0[2] ≥ 2 + 1.33·x[2]/x[1] + 1.33·x[3]/x[1] + 2 + 1.33·x[4]/x[2] + 1.33·x[5]/x[2] + 2 + 1e+03/x[5]), gpkit.PosynomialInequality(T0[2] ≤ 30.3), gpkit.PosynomialInequality(T0[3] ≥ 2 + 1.33·x[2]/x[1] + 1.33·x[3]/x[1] + 2 + 1.33·x[5

In [26]:
def minimum_area_slackBased(fanouts_dict, critical_paths_dict, slack):
  Twall = max(slack)
  slackList = [round(Twall-x,2) for x in slack]
  print(slackList)
  newList = []
  for i,s in enumerate(slackList):
    newList.append(slack[i]+s)
  print(Twall)
  N = len(fanouts_dict)
  Gsize = VectorVariable(N, "x", sign="positive")
  noofCP = len(all_paths)
  Tmin = VectorVariable(noofCP, "T0", sign = "positive")

  g = [1.0] * N
  p = [1.0] * N
  gate_index_map = {}
  for gate in fanouts_dict:
      gate_type, num_inputs, _ = get_gate_type_and_inputs(gate)
      idx = int(gate.split("_")[1]) - 1
      gate_index_map[gate] = idx
      g[idx] = logicalEffortDict.get((gate_type, num_inputs), 1.0)
      p[idx] = num_inputs

  constraints = []

  for i, path in enumerate(all_paths):
      delay_expr = 0
      for gate in path:
          idx = gate_index_map[gate]
          delay_expr += p[idx]
          for fanout in fanouts_dict[gate]:
              f_idx = gate_index_map[fanout]
              delay_expr += g[f_idx] * (Gsize[f_idx]) / Gsize[idx]
      last_gate_idx = gate_index_map[path[-1]]
      delay_expr += cload / Gsize[last_gate_idx]
      constraints.append(Tmin[i] >= delay_expr)
      constraints.append(Tmin[i] <= Twall)

  for gateName in inpGates:
      idx = gate_index_map[gateName]
      constraints.append(g[idx] * Gsize[idx] <= 50)

  constraints += [Gsize >= 1, Gsize <= max_gate_size]

  objective = sum(Gsize)
  m = Model(objective, constraints)
  sol = m.solve(verbosity=0)
  print(f"Minimum Area: {sol['cost']:.2f}")
  print("Optimized sizes:", list(f"{x:.2f}" for x in sol("x")))
  # print("Sizes:",list(float(sol["variables"]["x"])))

  print(max(sol["variables"]["T0"]))
  return sol['cost'], sol("x").tolist()
minimum_area_slackBased(fanouts_dict, critical_paths_dict, slack)

[0.0, 0.0, 0.0, 0.0]
28.832671043103176
Minimum Area: 219.18
Optimized sizes: ['9.27 dimensionless', '32.37 dimensionless', '33.03 dimensionless', '16.52 dimensionless', '64.00 dimensionless', '64.00 dimensionless']
28.832666562432614


(219.1847,
 [<Quantity(9.26763293, 'dimensionless')>,
  <Quantity(32.3659842, 'dimensionless')>,
  <Quantity(33.034022, 'dimensionless')>,
  <Quantity(16.517014, 'dimensionless')>,
  <Quantity(63.9999947, 'dimensionless')>,
  <Quantity(63.9999947, 'dimensionless')>])

In [ ]:
#branck and bound
Tspec = 1.3*max(slack)
N = len(fanouts_dict)
Gsize = VectorVariable(N, "x", sign="positive")
noofCP = len(all_paths)
Tmin = VectorVariable(noofCP, "T0", sign = "positive")

g = [1.0] * N
p = [1.0] * N
gate_index_map = {}
for gate in fanouts_dict:
    gate_type, num_inputs, _ = get_gate_type_and_inputs(gate)
    idx = int(gate.split("_")[1]) - 1
    gate_index_map[gate] = idx
    g[idx] = logicalEffortDict.get((gate_type, num_inputs), 1.0)
    p[idx] = num_inputs
constraints = []
constraints += [Gsize >= 1, Gsize <= max_gate_size]
for gateName in inpGates:
      idx = gate_index_map[gateName]
      constraints.append(g[idx] * Gsize[idx] <= 50)

def discreteSolution(fanouts_dict, critical_paths_dict, Tspec, alpha, newconstraint):
  constraints += newconstraint
  for i, path in enumerate(all_paths):
      delay_expr = 0
      for gate in path:
          idx = gate_index_map[gate]
          delay_expr += p[idx]
          for fanout in fanouts_dict[gate]:
              f_idx = gate_index_map[fanout]
              delay_expr += g[f_idx] * (Gsize[f_idx]) / Gsize[idx]
      last_gate_idx = gate_index_map[path[-1]]
      delay_expr += cload / Gsize[last_gate_idx]
      constraints.append(Tmin[i] >= delay_expr)
      constraints.append(Tmin[i] <= Tspec)


  objective = sum(Gsize)
  m = Model(objective, constraints)
  sol = m.solve(verbosity=0)
  print("Alpha: ",alpha)
  print(f"Minimum Area: {sol['cost']:.2f}")
  print("Optimized sizes:", list(f"{x:.1f}" for x in sol("x")))
  # print("Sizes:",list(float(sol["variables"]["x"])))

  print(max(sol["variables"]["T0"]))
  return sol['cost'], sol("x").tolist()